In [1]:
import sys 
sys.path.append('../')
import jax 
import jax.numpy as jnp 
import numpy as np
from jaxlie import SE3, SO3
from jax.random import normal
from jax import grad, hessian, vmap, pmap
from jax.flatten_util import ravel_pytree
from jax.lax import scan
from functools import partial
import jax.random as jax_random
import matplotlib.pyplot as plt 
from IPython.display import clear_output
from plyfile import PlyData
import open3d as o3d 
import trimesh as tm
import trimesh
import time
import jaxopt
import polyscope as ps


import meshcat
import meshcat.geometry as mc_geom
from robomeshcat import Object, Robot, Scene



from ergodic_mmd.aug_lagrange_jaxopt import AugmentedLagrangeSolver
# from ergodic_mmd.aug_lagrange_solver import AugmentedLagrangeSolver

import adam
from adam.jax import KinDynComputations

/Users/ia285/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
objc[10024]: Class GLFWApplicationDelegate is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x16b752e28) and /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x29ed0ee68). One of the two will be used. Which one is undefined.
objc[10024]: Class GLFWWindowDelegate is implemented in both /Users/ia285/miniconda3/lib/python3.10/site-packages/open3d/cpu/pybind.cpython-310-darwin.so (0x16b752db0) and /Users/ia285/miniconda3/lib/python3.10/site-packages/polyscope_bindings.cpython-310-darwin.so (0x29ed0ee90). One of the two will be used. Which one is undefined.
objc[10024]: Class GLFWContentView is implemented in both /Users

In [2]:
# dataset = o3d.data.BunnyMesh()
# plydata = PlyData.read(dataset.path)
# plydata = PlyData.read('../assets/sphere.ply')
plydata = PlyData.read('../assets/submarine.ply')
vertices = np.vstack((
    plydata['vertex']['x'],
    plydata['vertex']['y'],
    plydata['vertex']['z']
)).T

# _min_pnt = vertices.min(axis=0)
# _max_pnt = vertices.max()
# _mid_pnt = vertices.mean(axis=0)
# vertices = vertices/_max_pnt

# vertices = (vertices - _mid_pnt)/(_max_pnt-_min_pnt)

faces = np.array(np.vstack(plydata['face']['vertex_indices']))

mesh = tm.Trimesh(vertices=vertices, faces=faces)

mesh.apply_scale(0.001)
mesh.apply_transform(
    SE3.from_rotation(SO3.from_x_radians(np.pi/2)).as_matrix()
    )
mesh.apply_translation(np.array([0.,0.,0.25]))
# ds_mesh = mesh
# ds_mesh = mesh.simplify_quadric_decimation(int(mesh.vertices.shape[0]/10))


# ds_mesh = tm.primitives.Box()
ds_mesh = mesh.convex_hull

In [3]:
scene = Scene()
exp_obj = trimesh.exchange.obj.export_obj(mesh)
mc_obj = mc_geom.ObjMeshGeometry.from_stream(trimesh.util.wrap_as_stream(exp_obj))
mc_obj = Object(mc_obj, name='red_sphere', opacity=1, color=[.5, .5, .5])
# exp_obj = trimesh.exchange.obj.export_obj(ds_mesh)
# mc_obj = mc_geom.ObjMeshGeometry.from_stream(trimesh.util.wrap_as_stream(exp_obj))
# mc_obj = Object(mc_obj, name='red_sphere', opacity=0.5, color=[1., 0., 0.])
scene.add_object(mc_obj)

# bluerov = Object.create_mesh("../assets/blue_rov2.stl", name="bluerov", color=[0.,0.,1.0])
# scene.add_object(bluerov)


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/


In [4]:
num_points = 200  # Change this number based on your requirement
points, face_indices = tm.sample.sample_surface(ds_mesh, num_points, seed=0)

# Get the normals for each sampled point
normals = ds_mesh.face_normals[face_indices]

info_distr = lambda x: 1.0
P_XI = vmap(info_distr, in_axes=(0,))(points)
P_XI = P_XI/jnp.sum(P_XI)
h = 0.1
args = {'h' : h, 
        'points' : points+0.1*ds_mesh.face_normals[face_indices], 
        'normals' : ds_mesh.face_normals[face_indices], 
        'P_XI' : P_XI}

In [5]:
def RBF_kernel(x, xp, h=0.01):
    return jnp.exp(
        -jnp.sum((x-xp)**2)/h
    )
    # * jnp.exp(
    #     -1e-3*jnp.sum((norm-w)**2)/h
    # ) 
    # return jnp.exp(
    #     -jnp.sum((x-xp)**2)/h
    # )
# def camera_view_penalty(x, xp, h=0.01):
#     w, v        = jnp.split(x,2)
#     norm, root  = jnp.split(xp, 2)
#     return jnp.exp(
#         -jnp.sum((root-v)**2)/h #- 1e-1*jnp.sum((norm-w)**2)
#     ) * jnp.exp(
#         -1e-1*jnp.sum((norm-w)**2)
#     )

def camera_view_penalty(g, v, h=0.01):
    p = g.translation()
    rot = g.rotation()
    w = rot.apply(jnp.array([0.,0.,1.]))
    root, norm  = jnp.split(v, 2)
    # return jnp.exp(-jnp.sum((root-p)**2)/h - 0.01*jnp.sum((-norm-w)**2)/h)
    return jnp.exp(
        -jnp.sum((root-p)**2)/h #- 1e-1*jnp.sum((norm-w)**2)
    )*jnp.dot(-norm,w)
    # * jnp.exp(
    #     # -jnp.sum((-norm-w)**2)/h
    #     jnp.dot(-norm,w)
    # )

def create_kernel_matrix(kernel):
    return vmap(vmap(kernel, in_axes=(0, None, None)), in_axes=(None, 0, None))

In [6]:
KernelMatrix = create_kernel_matrix(RBF_kernel)
camera_view_matrix = create_kernel_matrix(camera_view_penalty)


def emmd_loss(params, args):
    X = params['X']
    T = X.shape[0]

    g = vmap(SE3.exp)(X)
    p = g.translation()

    h = args['h']
    points    = args['points']
    norms     = args['normals']
    P_XI      = args['P_XI']

    view_matrix = camera_view_matrix(g, jnp.hstack([points, norms]), h)

    return np.sum(KernelMatrix(p, p, h))/(T**2) \
            - 2 * np.sum(P_XI @ KernelMatrix(p, points, h))/T \
            + np.mean((X[1:]-X[:-1])**2) - np.sum(P_XI@view_matrix)/T
            # + np.sum((X_init[0]-x0)**2)

    # view_matrix = camera_view_matrix(X_init, XP, h)
    # x0 = args['x0']
    # vertices = args['vertices']
    # norms    = args['normals']
    # XP = jnp.hstack([norms, vertices])
    # P_XI      = jnp.sum(phi_vmapped(X_init, XP, h), axis=1)
    # return np.sum(KernelMatrix(X_init, X_init, h))/(T**2) \
    #         - 2 * np.sum(P_XI @ KernelMatrix(X_init, XP, h))/T \
    #         + np.mean((v[1:]-v[:-1])**2) - np.sum(P_XI@view_matrix)/T
    #         # + np.sum((X_init[0]-x0)**2)

def eq_constr(params, args):
    X = params['X']
    # q0 = args['q0']
    return jnp.zeros(1)#jnp.hstack([X[0]-q0, X[-1]-q0])

def ineq_constr(params, args):
    X = params['X']
    return jnp.zeros(1)#jnp.square(X[1:] - X[:-1]) - 0.01**2

In [7]:
T = 100
X = jnp.linspace(-0.1*jnp.ones(6), 0.1*jnp.ones(6), num=T)

params = {'X' : X}

# solver = AugmentedLagrangeSolver(params, emmd_loss, eq_constr, ineq_constr, max_stepsize=3e-1, args=args)
solver = AugmentedLagrangeSolver(params, emmd_loss, eq_constr, ineq_constr, args=args)

In [8]:
solver.solve(max_iter=50_000, eps=1e-4)

0 0.22673378507438494
1 0.2115011023757943
2 0.19100761261195773
3 0.17040521305964976
4 0.15225369081766316
5 0.13730315851361077
6 0.12541022646211855
7 0.11609964882581802
8 0.10883767393954581
9 0.10314633892413069
10 0.09864049912570896
11 0.09502952129395172
12 0.09210416583679915
13 0.08971867314890819
14 0.08777281986875912
15 0.08619603749772772
16 0.0849343703061317
17 0.08394046702795084
18 0.08316658375950431
19 0.08256049653027726
20 0.08206414918919534
21 0.08161473647181569
22 0.08114774142625358
23 0.08060124966856812
24 0.07992071148665515
25 0.07906327945801336
26 0.07800095555077446
27 0.07672203716975833
28 0.07523070508330208
29 0.07354495860732845
30 0.07169338106538016
31 0.06971135002672159
32 0.06763728397559998
33 0.06550937952429114
34 0.06336310392404261
35 0.06122952504071343
36 0.059134422281452985
37 0.05709803966612421
38 0.05513531060387445
39 0.0532563884464771
40 0.0514673419037941
41 0.049770907378353484
42 0.04816722301347394
43 0.04665449742782447


KeyboardInterrupt: 

In [61]:
def create_robot_render(name):
    blue_rov = trimesh.load_mesh('../assets/blue_rov_reduced2.STL')
    blue_rov.apply_scale(0.0001)

    _bb = blue_rov.bounds

    blue_rov.apply_translation((_bb[0]-_bb[1])/2)
    # blue_rov.apply_transform(
    #     SE3.from_rotation(SO3.from_x_radians(np.pi/2)).as_matrix()
    #     )

    blue_rov = trimesh.exchange.obj.export_obj(blue_rov)
    blue_rov = mc_geom.ObjMeshGeometry.from_stream(trimesh.util.wrap_as_stream(blue_rov))
    blue_rov = Object(blue_rov, name=name,  color=[0.2, 0.2, .9])
    return blue_rov

# scene.add_object(create_robot_render("blue_rov1"))


In [62]:
sol = solver.solution
X = sol['X']

vertices = vmap(SE3.exp)(X).translation().astype(np.float32).T
scene.vis['lines_segments'].set_object(mc_geom.Line(mc_geom.PointsGeometry(vertices), mc_geom.MeshLambertMaterial(color=0xff0000, linewidth=2)))


robot_img = []
for i, q in enumerate(X[::int(T/10)]):
    # _tf = np.array(get_fk(q).as_matrix()).astype(np.float64)
    # ee_frame.set_transform(_tf)
    ee_frame = scene.vis['robot{}_ee'.format(i)]
    ee_frame.set_object(mc_geom.triad(scale=0.1))

    _robot = create_robot_render('robot{}'.format(i))
    robot_img.append(_robot)
    scene.add_object(_robot)
    _tf = np.array(SE3.exp(q).as_matrix()).astype(np.float64)
    _robot.pose = _tf
    ee_frame.set_transform(_tf)

"Render the initial scene"
scene.render()

In [83]:
sol = solver.solution
X = sol['X']
vertices = vmap(SE3.exp)(X).translation().astype(np.float32).T
scene.vis['lines_segments'].set_object(
        mc_geom.Line(mc_geom.PointsGeometry(vertices), 
                     mc_geom.MeshLambertMaterial(color=0xff0000),
                     
                )
            )
ee_frame = scene.vis['ee']
ee_frame.set_object(mc_geom.triad(scale=0.2))
for q in X:
    _tf = np.array(SE3.exp(q).as_matrix()).astype(np.float64)
    ee_frame.set_transform(_tf)
    time.sleep(0.1)
    scene.render()

In [8]:
ps.init()

ps_bunny_mesh = ps.register_surface_mesh("bunny", ds_mesh.vertices, ds_mesh.faces)
ps_bunny_pcl = ps.register_point_cloud("bunny points", points+normals*0.2)


ps_bunny_mesh.set_transparency(0.8)
ps_traj         = ps.register_curve_network("trajectory", X_init * 1.25, edges="line")

for _ in range(1000):
    (sol, solver_state) = solver.update(
                        params=sol, 
                        state=solver_state, 
                        hyperparams_proj=bounds, 
                                args=args)
    X = unflatten_X(sol)
    ps_traj.update_node_positions(X[:,3:])
    ps_traj.add_vector_quantity("vec img", X[:,:3], enabled=True)
    ps.frame_tick()

    time.sleep(0.001)

[polyscope] Backend: openGL3_glfw -- Loaded openGL version: 4.1 Metal - 88


In [8]:
ps.init()

ps_bunny_mesh = ps.register_surface_mesh("bunny", ds_mesh.vertices, ds_mesh.faces)
ps_bunny_mesh.set_transparency(0.8)
ps_traj         = ps.register_curve_network("trajectory", X_init * 1.25, edges="line")

# ps_bunny_mesh.add_scalar_quantity('info_distr', mesh_func.func_vals, defined_on='vertices', cmap='blues')

ps_traj.update_node_positions(X[:,3:])
ps_traj.add_vector_quantity("vec img", X[:,:3], enabled=True)


for i,_xi in enumerate(X): 
    _dir, _root =  np.split(_xi, 2)
    intrinsics = ps.CameraIntrinsics(fov_vertical_deg=60, aspect=2)

    extrinsics = ps.CameraExtrinsics(root=np.array(_root), look_dir=np.array(_dir), up_dir=(0.,1.,0.))
    params = ps.CameraParameters(intrinsics, extrinsics)
    cam = ps.register_camera_view('cam{}'.format(i), params)
ps.show()

KeyboardInterrupt: 

In [ ]:
def lie_RBF(x, xp, h=0.01):
    w, v        = jnp.split(x,2)
    norm, root  = jnp.split(xp, 2)
    _R = SO3.exp(w)
    return jnp.exp(
        -(jnp.sum((norm+_R@v1)**2) + np.sum((norm+root-v)**2))/h
    )

def create_kernel_matrix(kernel):
    return vmap(vmap(kernel, in_axes=(0, None, None)), in_axes=(None, 0, None))